# DevMuT Artifact

Artifact of the paper "DevMuT: Testing Deep Learning Framework via Developer
Expertise-Based Mutation" from ASE 2024.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

In [ ]:
from chi import context

context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

In [ ]:
from chi import hardware

node_type = "gpu_p100"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

### Reserve node

In [ ]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

### Create a server on the node

In [ ]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

### Configure networking on the node

In [ ]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

## Run DevMuT

Now, we can finally run DevMuT. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

Install Nvidia Drivers

In [ ]:
script = """
#!/bin/bash

# Step 1: Update the system
sudo apt update && sudo apt upgrade -y

# Step 2: Add NVIDIA package repositories
sudo apt install -y software-properties-common
sudo add-apt-repository ppa:graphics-drivers/ppa -y
sudo apt update

# Step 3: Install the latest NVIDIA driver (replace "525" with the desired version if needed)
sudo apt install -y nvidia-driver-525
"""

my_server.execute(script)

In [ ]:
my_server.execute("git clone https://github.com/radhofan/DevMuT.git")

In [ ]:
my_server.execute("chmod +x DevMuT/reproduce.sh")
my_server.execute("bash DevMuT/reproduce.sh")